In [ ]:
##!/usr/bin/env python
"""plot_skill_lsm.py

Script plots the normalised error scores for WRF-LIS-CABLE physics configurations and WRF-Noah CORDEX simulations

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu May  2 12:15:30 AEST 2019

"""

In [1]:
# Load packages

#from __future__ import division
import numpy as np
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
import common_functions as cf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# WRF-LIS-CABLE Simulation Details

sims=["PHYS_TEST_RA1_PBL2_CU1", "PHYS_TEST_RA1_PBL2_CU2", "PHYS_TEST_RA1_PBL1_CU1"]


In [3]:
rlabels=["CABLE 1"," CABLE 2","CABLE 3","Noah 1","Noah 2", "Noah 3"]

In [4]:
# CORDEX land-sea mask

# Landsea mask
cordexmaskfile     ='/g/data3/w97/azh561/analysis/cordex_australasia/scripts/geo_em_files/geo_em.AUS-44i.nc'
cordexfilemask = nc.Dataset(cordexmaskfile)
cordexmask     = cordexfilemask.variables['LANDMASK'][:]
cordexmask     = cordexmask.astype(int)  
cordexlat      = cordexfilemask.variables['lat'][:]
cordexlon      = cordexfilemask.variables['lon'][:]

# If the longitude spans [-180 180] then update to that instead it is [0 360] - better of AUS domain
if np.min(cordexlon) < 0.0:
    print('Longitude values span [-180 180] will change to [0 360]')
    for jj in range(len(cordexlon)):
        if cordexlon[jj] < 0.0:
            cordexlon[jj]=cordexlon[jj]+360

Longitude values span [-180 180] will change to [0 360]


In [5]:
# UNSW WRF-Noah Simulation Details

unswmaindir = "/g/data3/hh5/tmp/w28/gdv561/CORDEX/ERA-Int/ERA-Interim_post/postprocess/CORDEX/output/AUS-44i/UNSW/ECMWF-ERAINT/evaluation/r1i1p1/"
unswmiddir = "/v1/d01/day/"
unswrcms = ["UNSW-WRF360J", "UNSW-WRF360K"]
var = ["pr","tasmax","tasmin","hfls"]
unswprefix = "_AUS-44i_ECMWF-ERAINT_evaluation_r1i1p1_"
unswendfix = "_v1_day_20060101-20101231.nc"
unswst = [1066,1066,1157,1249,1341]
unswfi = [1431,1156,1248,1340,1431]


In [6]:
# MU WRF-Noah Simulation

mumaindir = "/g/data3/w97/azh561/cordex_australasia/MU-WRF/AUS-44i/MU-WRF330/day/" 
muendfix = "_AUS-44i_ECMWF-ERAINT_evaluation_r1i1p1_MU-WRF330_v1_day_20050101-20091231.nc"
must = [1431,1431,1522,1614,1706]
mufi = [1796,1521,1613,1705,1796]


In [7]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')

# Data directory 
datadir = '/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(rlabels)
obs = "AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='LSM_SKILL_AWAP_MASKED'

if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
variables=['PR','TX','TN','QE']
nvar=len(variables)

# Define regions to zoom in on
latN=[-23.43,-38.0,-34.0]
latX=[-10.0,-27.0,-30.0]
lonN=[110.0,139.0,114.0]
lonX=[155.0,151.0,120.0]
Rname=["TROPICS","SEA","SWWA"]
nreg = len(Rname)

# Define the mean climate intervals of interest    
mclim=["ANN","DJF","MAM","JJA","SON"]
ytarget=["2009"]
mtarget=[["12","01","02"],["03","04","05"],["06","07","08"],["09","10","11"]]
nmean = len(mclim)


vlabels=['$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$',
         '$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$',
         '$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$']



In [8]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

In [9]:
sbias_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
srmse_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sphase_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sdsn_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)

In [10]:
# Load observational data on the WRF-LIS-CABLE grid

filename='%s/%s_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip = file.variables['awap_precip_mask'][:]
obst2max = file.variables['awap_t2max_mask'][:]
obst2min = file.variables['awap_t2min_mask'][:]
file.close()

filename='%s/GLEAM_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls = file.variables['gleam_hfls_mask'][:]
file.close()

ndays = 365
sind = 336

# Truncate data to simulation period - removing the spinup aswell
obsprecipT = obsprecip[sind:sind+ndays,:,:]
obst2maxT = obst2max[sind:sind+ndays,:,:]
obst2minT = obst2min[sind:sind+ndays,:,:]
obshflsT = obshfls[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma = np.ma.masked_array(obsprecipT, obsprecipT>=1.e20).filled(np.nan)
obst2maxma  = np.ma.masked_array(obst2maxT, obst2maxT>=1.e20).filled(np.nan)
obst2minma  = np.ma.masked_array(obst2minT, obst2minT>=1.e20).filled(np.nan)
obshflsma  = np.ma.masked_array(obshflsT, obst2maxT>=1.e20).filled(np.nan)


In [11]:
# WRF-CABLE Skill

# Loop through the experiments
for mind,mname in enumerate(sims):

    # Load precip data - excluding 2 month spinup
    filename='%s/%s_WRF_Precip_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    precip = file.variables['wrf_precip_mask'][123:488,:,:]
    file.close()

    # Load temperature data - excluding 2 month spinup
    filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    t2max = file.variables['T2MAX'][123:488,:,:]
    t2min = file.variables['T2MIN'][123:488,:,:]
    time = nc.chartostring(file.variables['Times'][123:488,:])
    timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
    years = np.asarray([timesplit[x].split("-")[0] for x in range(len(time))])
    months = np.asarray([timesplit[x].split("-")[1] for x in range(len(time))])
    file.close()

    # Load hfls data - excluding 2 month spinup
    filename='%s/%s_lisout_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    hfls = file.variables['lis_hfls_mask'][123:488,:,:]
    lat2d = file.variables['wrf_lat2d'][:]
    lon2d = file.variables['wrf_lon2d'][:]
    file.close()
    
    # Loop through the regions
    for rind,rname in enumerate(Rname):
        
        # Get lat/lon indices to truncate data to region of interest
        bbox = [lonN[rind],lonX[rind], latN[rind], latX[rind]]
        i0,i1,j0,j1 = cf.bbox2ij(lon2d,lat2d,bbox)
        
        # Truncate to the period of interest for climate means
        for pind,pname in enumerate(mclim):
            
            # Get relevant time indices
            if pname in ["ANN"]:
                tind = np.arange(0,len(time),1)
            else:
                tind = np.asarray([months[x] in mtarget[pind-1] for x in range(len(months))])

            # Calculate the bias score
            sbias_mean[mind,rind,pind,0] = cf.calc_bias(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,1] = cf.calc_bias(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,2] = cf.calc_bias(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,3] = cf.calc_bias(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the rmse score
            srmse_mean[mind,rind,pind,0] = cf.calc_rmse(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,1] = cf.calc_rmse(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,2] = cf.calc_rmse(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,3] = cf.calc_rmse(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the phase shift score
            sphase_mean[mind,rind,pind,0] = cf.calc_phase(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,1] = cf.calc_phase(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,2] = cf.calc_phase(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,3] = cf.calc_phase(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the spatial dsn score
            sdsn_mean[mind,rind,pind,0] = cf.calc_spatialdsn(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,1] = cf.calc_spatialdsn(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,2] = cf.calc_spatialdsn(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,3] = cf.calc_spatialdsn(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            del tind
        
        del bbox,i0,i1,j0,j1
    
    del precip, t2max, t2min, hfls
    del time,timesplit,years,months


/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:42: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:81: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/numpy/ma/core.py:6666: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:151: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)


In [12]:
# Load observational data on the WRF-Noah grid

filename='%s/%s_NOAH_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip2 = file.variables['awap_precip_mask'][:]
obst2max2 = file.variables['awap_t2max_mask'][:]
obst2min2 = file.variables['awap_t2min_mask'][:]
file.close()

filename='%s/GLEAM_NOAH_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls2 = file.variables['gleam_hfls_mask'][:]
file.close()

ndays = 365+1
sind = 336

# Truncate data to simulation period - removing the spinup aswell
obsprecipT2 = obsprecip2[sind:sind+ndays,:,:]
obst2maxT2 = obst2max2[sind:sind+ndays,:,:]
obst2minT2 = obst2min2[sind:sind+ndays,:,:]
obshflsT2 = obshfls2[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma2 = np.ma.masked_array(obsprecipT2, obsprecipT2>=1.e20).filled(np.nan)
obst2maxma2  = np.ma.masked_array(obst2maxT2, obst2maxT2>=1.e20).filled(np.nan)
obst2minma2  = np.ma.masked_array(obst2minT2, obst2minT2>=1.e20).filled(np.nan)
obshflsma2  = np.ma.masked_array(obshflsT2, obst2maxT2>=1.e20).filled(np.nan)


obst=[0,0,91,183,275]
obfi=[365,90,182,274,365]

In [13]:
# WRF-CABLE Noah

# Read in the Noah simulations
for mind in range(3):   
    for vind in range(4):

        # Observational data
        if vind == 0:
            odata = obsprecipma2
        elif vind == 1:
            odata = obst2maxma2
        elif vind == 2:
            odata = obst2minma2
        elif vind == 3:
            odata = obshflsma2
            
        # Load data
        if mind < 2: # UNSW WRF-Noah simulations
            filename = '%s/%s/%s/%s/%s%s%s%s' %(unswmaindir,unswrcms[mind],unswmiddir,var[vind],var[vind],unswprefix,unswrcms[mind],unswendfix)
            stind = unswst
            fiind = unswfi
        else: # MU WRF-Noah Simulation
            filename = '%s/%s/%s%s' %(mumaindir,var[vind],var[vind],muendfix)
            stind = must
            fiind = mufi  

        # Load data
        file = nc.Dataset(filename)
        if vind == 0:
            if mind < 2:
                data = file.variables[var[vind]][:] * 86400 # Convert to mm/day
            else:
                data = file.variables[var[vind]][:]
        elif vind == 3:
            data = file.variables[var[vind]][:] 
        else:
            data = file.variables[var[vind]][:] - 273.15 # Convert to C

        file.close()
        
        # Mask data according to the landsea-mask
        lsmask3d = np.repeat(cordexmask[np.newaxis],data.shape[0],axis=0)
        datama = np.ma.masked_where(lsmask3d!=1.,data)
        olsmask3d = np.repeat(cordexmask[np.newaxis],odata.shape[0],axis=0)
        odatama = np.ma.masked_where(olsmask3d!=1.,odata)
        
        datama = xarray.DataArray(datama, coords=[('time',np.arange(0,datama.shape[0])),('cordexlat', cordexlat), ('cordexlon', cordexlon)])
        odatama = xarray.DataArray(odatama, coords=[('time',np.arange(0,odatama.shape[0])),('cordexlat', cordexlat), ('cordexlon', cordexlon)])

        # loop through the regions
        for rind,rname in enumerate(Rname):
            
            # Truncate data to region 
            latT = cordexlat[(cordexlat>=latN[rind]) & (cordexlat<=latX[rind])]
            lonT = cordexlon[(cordexlon>=lonN[rind]) & (cordexlon<=lonX[rind])]
            clon2d,clat2d = np.meshgrid(lonT,latT)
            dataT = datama.sel(cordexlat = slice(latN[rind], latX[rind]), cordexlon = slice(lonN[rind], lonX[rind]))
            odataT = odatama.sel(cordexlat = slice(latN[rind], latX[rind]), cordexlon = slice(lonN[rind], lonX[rind]))
        
            # Truncate to the period of interest for climate means
            for pind,pname in enumerate(mclim):
            
                # Calculate the bias score
                sbias_mean[mind+3,rind,pind,vind] = cf.calc_bias(dataT[stind[pind]:fiind[pind],:,:],odataT[obst[pind]:obfi[pind],:,:],clat2d)

                # Calculate the rmse score
                srmse_mean[mind+3,rind,pind,vind] = cf.calc_rmse(dataT[stind[pind]:fiind[pind],:,:],odataT[obst[pind]:obfi[pind],:,:],clat2d)

                # Calculate the phase shift score
                sphase_mean[mind+3,rind,pind,vind] = cf.calc_phase(dataT[stind[pind]:fiind[pind],:,:],odataT[obst[pind]:obfi[pind],:,:],clat2d)

                # Calculate the spatial dsn score
                sdsn_mean[mind+3,rind,pind,vind] = cf.calc_spatialdsn(dataT[stind[pind]:fiind[pind],:,:],odataT[obst[pind]:obfi[pind],:,:],clat2d)

            del dataT, odataT, latT, lonT, clat2d, clon2d
            
        del data, datama, odata, odatama, lsmask3d, olsmask3d
     

/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:41: RuntimeWarning: Mean of empty slice
  mmean = np.nanmean(mdata,axis=0)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:80: RuntimeWarning: Mean of empty slice
  mmean = np.nanmean(mdata,axis=0)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:150: RuntimeWarning: Mean of empty slice
  mmean = np.nanmean(mdata,axis=0)


In [14]:
# Calculate the overall score - climate
s_mean = (sbias_mean + 2*srmse_mean + sphase_mean + sdsn_mean)/5.

In [15]:
# Function to create plot
def plot_SKILL(scores,RCMlabels,figurename,vlabels,Rname,mtitle,mx=None,mn=None):
    """This function plots model skill in the context of observational uncertainty.
    
    Input arguments: 
        scores - multi-dimensional array containing the skill scores
        RCMlabels - list containing model/experiment names
        figurename - file name for saving the figure to
        vlabels - array containing the variables included
        Rname - region names
        mtitle - main title
        mx = max value on colorbar
        mn = min value on colorbar
    """

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    from matplotlib.colors import LinearSegmentedColormap

    # Retrieve dimensions
    nmod = scores.shape[0] # Models
    nreg = scores.shape[1] # Regions
    nper = scores.shape[2] # Time periods
    nvar = scores.shape[3] # Variables
        
    # Create figure object and subplots
    plt.rcParams['savefig.dpi'] = 1000
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    # Create figure object and subplots
    nrow=1
    ncol=2

    gs = mpl.gridspec.GridSpec(nrow,ncol, width_ratios=[2.,0.5], wspace=0.05)
    fig = plt.figure(figsize=(4,16))

    # Make axes
    ax1 = fig.add_subplot(gs[0,0])

    # Colour bar axes (':' as the colour bars cover multiple rows)
    # Use a new subplot so we can control the spacing better
    cgs = mpl.gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=gs[:,-1], wspace=2)
    cax1 = plt.subplot(cgs[0,0]) 

    if not mx:
        mx=1.
    if not mn:
        mn=0.
    
    # Define the colormap - using a 'traffic light' approach
    nbins=20    
    cmap = plt.get_cmap('RdYlGn')
    cmap.set_bad('w')
    levels = MaxNLocator(nbins=nbins).tick_values(mn,mx)
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    # Reshape from 4D to 2D array to plot the skill scores as rectanges
    data = np.empty((nreg*nper*nvar,nmod),dtype=np.float64)
    for rr in range(nreg):
        for vv in range(nvar):
            for pp in range(nper):
                data[(rr*nvar*nper) + (vv*nper) + pp,:] = scores[:,rr,pp,vv]

    xarr = np.arange(0.5,nmod+1,1)
    xticks = np.arange(1,nmod+1,1)
    yarr = np.arange(0.5,nreg*nper*nvar+1,1)
    yticks = np.arange(1,nreg*nper*nvar+1,1)
    
    # Plot the skill scores   
    skill = ax1.pcolormesh(xarr,yarr[::-1],data,cmap=cmap,norm=norm)
    ax1.set_yticklabels(vlabels[::-1],ha='right')
    ax1.set_yticks(yticks)
    ax1.set_ylim(yarr[0],yarr[-1])
    ax1.set_xlim(xarr[0],xarr[-1])
    ax1.set_xticklabels(RCMlabels, fontsize = 8, rotation = 90.)
    ax1.set_xticks(xticks)

    for x in range(len(xticks)-1):
        ax1.axvline(x=xticks[x]+0.5, color='black', linestyle='--',linewidth=1.0)

    ax1.axhline(yticks[nvar*nper]-0.5, color='black', linestyle='-',linewidth=1.0)
    ax1.axhline(yticks[2*nvar*nper]-0.5, color='black', linestyle='-',linewidth=1.0)

    ax1.text(-2, 53, Rname[0], fontsize=14, rotation = 90.)
    ax1.text(-2, 31, Rname[1], fontsize=14, rotation = 90.)
    ax1.text(-2, 12, Rname[2], fontsize=14, rotation = 90.)
    
    ax1.set_title('%s' %(mtitle), fontweight='bold')
    fig.colorbar(skill,cax1,ticks=levels[::2])
    
    # Finalise the figure layout
    
    fig.tight_layout()   
    fig.subplots_adjust(wspace=0, hspace=0, left=0.25)

    fig.savefig(figurename,bbox_inches = "tight")
    plt.close(fig)


# Plot the Scalar Skill Scores

In [16]:
# Plot the bias skill score
figurename='%s/%s_BIAS.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sbias_mean,rlabels,figurename,vlabels,Rname,'Bias Skill Score')

# Plot the RMSE skill score
figurename='%s/%s_RMSE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(srmse_mean,rlabels,figurename,vlabels,Rname,'RMSE Skill Score')

# Plot the Phase shift skill score
figurename='%s/%s_PHASE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sphase_mean,rlabels,figurename,vlabels,Rname,'Phase Shift Skill Score')

# Plot the Spatial Distribution skill score
figurename='%s/%s_SDSN.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sdsn_mean,rlabels,figurename,vlabels,Rname,'Spatial Distribution Skill Score')

# Plot the Overall skill score
figurename='%s/%s_ALL.png' %(fig_dir,fig_name_prefix)
plot_SKILL(s_mean,rlabels,figurename,vlabels,Rname,'Overall Skill Score')


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/matplotlib/figure.py:2369: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [ ]:
#Rearrange x-axis order
rlabels=["CABLE 1","Noah 1"," CABLE 2","Noah 2","CABLE 3", "Noah 3"]

sbias_mean2 = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
srmse_mean2 = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sphase_mean2 = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sdsn_mean2 = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
s_mean2 = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)

sbias_mean2[0,:,:] = sbias_mean[0,:,:]
sbias_mean2[1,:,:] = sbias_mean[3,:,:]
sbias_mean2[2,:,:] = sbias_mean[1,:,:]
sbias_mean2[3,:,:] = sbias_mean[4,:,:]
sbias_mean2[4,:,:] = sbias_mean[2,:,:]
sbias_mean2[5,:,:] = sbias_mean[5,:,:]

srmse_mean2[0,:,:] = srmse_mean[0,:,:]
srmse_mean2[1,:,:] = srmse_mean[3,:,:]
srmse_mean2[2,:,:] = srmse_mean[1,:,:]
srmse_mean2[3,:,:] = srmse_mean[4,:,:]
srmse_mean2[4,:,:] = srmse_mean[2,:,:]
srmse_mean2[5,:,:] = srmse_mean[5,:,:]

sphase_mean2[0,:,:] = sphase_mean[0,:,:]
sphase_mean2[1,:,:] = sphase_mean[3,:,:]
sphase_mean2[2,:,:] = sphase_mean[1,:,:]
sphase_mean2[3,:,:] = sphase_mean[4,:,:]
sphase_mean2[4,:,:] = sphase_mean[2,:,:]
sphase_mean2[5,:,:] = sphase_mean[5,:,:]

sdsn_mean2[0,:,:] = sdsn_mean[0,:,:]
sdsn_mean2[1,:,:] = sdsn_mean[3,:,:]
sdsn_mean2[2,:,:] = sdsn_mean[1,:,:]
sdsn_mean2[3,:,:] = sdsn_mean[4,:,:]
sdsn_mean2[4,:,:] = sdsn_mean[2,:,:]
sdsn_mean2[5,:,:] = sdsn_mean[5,:,:]

s_mean2[0,:,:] = s_mean[0,:,:]
s_mean2[1,:,:] = s_mean[3,:,:]
s_mean2[2,:,:] = s_mean[1,:,:]
s_mean2[3,:,:] = s_mean[4,:,:]
s_mean2[4,:,:] = s_mean[2,:,:]
s_mean2[5,:,:] = s_mean[5,:,:]

figurename='%s/%s_BIAS_ORDER.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sbias_mean2,rlabels,figurename,vlabels,Rname,'Bias Skill Score')

# Plot the RMSE skill score
figurename='%s/%s_RMSE_ORDER.png' %(fig_dir,fig_name_prefix)
plot_SKILL(srmse_mean2,rlabels,figurename,vlabels,Rname,'RMSE Skill Score')

# Plot the Phase shift skill score
figurename='%s/%s_PHASE_ORDER.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sphase_mean2,rlabels,figurename,vlabels,Rname,'Phase Shift Skill Score')

# Plot the Spatial Distribution skill score
figurename='%s/%s_SDSN_ORDER.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sdsn_mean2,rlabels,figurename,vlabels,Rname,'Spatial Distribution Skill Score')

# Plot the Overall skill score
figurename='%s/%s_ALL_ORDER.png' %(fig_dir,fig_name_prefix)
plot_SKILL(s_mean2,rlabels,figurename,vlabels,Rname,'Overall Skill Score')